In [ ]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [ ]:
ROOT_DIR = os.path.dirname(os.getcwd())
DATA_FOLDER = os.path.join(ROOT_DIR, "data")

In [ ]:
image_path = os.path.join(DATA_FOLDER, "mercator.png")

In [ ]:
def custom_field_projection(img, scale_factor=1.0, compression_factor=0.8):
    """
    Custom projection with enhanced compression effect for football field panoramas.
    """
    height, width = img.shape[:2]
    
    # Create coordinate grid
    x = np.linspace(-1, 1, width)
    y = np.linspace(-1, 1, height)
    xv, yv = np.meshgrid(x, y)
    
    # Initialize mapping arrays
    map_x = np.zeros((height, width), dtype=np.float32)
    map_y = np.zeros((height, width), dtype=np.float32)
    
    # Enhanced vertical compression
    # Apply stronger effect to the top of the image
    y_normalized = yv.copy()
    
    # Make compression more dramatic
    power = 1.0 + (1.0 - compression_factor) * 2  # This will give more range
    y_compressed = np.sign(y_normalized) * np.power(np.abs(y_normalized), power)
    
    # Apply additional vertical stretching to top portion
    top_mask = y_normalized < 0
    y_compressed[top_mask] *= (1 + (1-compression_factor))
    
    # Convert back to pixel coordinates
    map_x = ((xv + 1) * width / 2).astype(np.float32)
    map_y = ((y_compressed + 1) * height / 2).astype(np.float32)
    
    # Ensure coordinates are within bounds
    map_x = np.clip(map_x, 0, width - 1)
    map_y = np.clip(map_y, 0, height - 1)
    
    # Verify shapes match before remapping
    assert map_x.shape == map_y.shape, "Map shape mismatch"
    assert map_x.shape == (height, width), f"Map shape {map_x.shape} doesn't match image shape {(height, width)}"
    
    try:
        projected = cv2.remap(img, map_x, map_y, cv2.INTER_LINEAR)
        return projected
    except cv2.error as e:
        print(f"Error during remapping: {e}")
        print(f"Image shape: {img.shape}")
        print(f"Map_x shape: {map_x.shape}")
        print(f"Map_y shape: {map_y.shape}")
        raise

In [ ]:
def visualize_projection(input_image_path, compression_factors=[0.3, 0.5, 0.7, 0.9]):
    """
    Visualize different compression factors side by side.
    """
    img = cv2.imread(input_image_path)
    if img is None:
        raise ValueError(f"Could not read input image: {input_image_path}")
    
    # Create a figure with subplots
    results = []
    for factor in compression_factors:
        projected = custom_field_projection(img, compression_factor=factor)
        results.append(projected)
    
    # Stack images horizontally and vertically to create a grid
    n = len(compression_factors)
    rows = []
    for i in range(0, n, 2):
        if i + 1 < n:
            row = np.hstack([results[i], results[i+1]])
        else:
            row = results[i]
        rows.append(row)
    grid = np.vstack(rows)
    
    # Add text labels
    font = cv2.FONT_HERSHEY_SIMPLEX
    y_offset = 30
    for i, factor in enumerate(compression_factors):
        col = i % 2
        row = i // 2
        x = col * img.shape[1] + 10
        y = row * img.shape[0] + y_offset
        cv2.putText(grid, f'Compression: {factor}', (x, y), font, 1, (0, 255, 0), 2)
    
    return grid

In [ ]:
visualization = visualize_projection(image_path)

In [ ]:
plt.figure(figsize=(30, 30))
plt.imshow(visualization)
plt.show()

In [ ]:
class CustomProjection:
    def __init__(self, vertical_stretch=1.2, center_weight=0.7, preserve_top=True):
        """
        Custom projection that combines aspects of transverse Mercator with additional controls
        
        Parameters:
        vertical_stretch: Factor to stretch the vertical dimension (>1 stretches, <1 compresses)
        center_weight: Weight for center region preservation (0-1)
        preserve_top: Whether to preserve top region of image
        """
        self.vertical_stretch = vertical_stretch
        self.center_weight = center_weight
        self.preserve_top = preserve_top

    def create_maps(self, img_shape):
        """
        Create x and y mapping matrices for the projection
        """
        height, width = img_shape[:2]
        
        # Create base coordinate grid
        x = np.linspace(0, width-1, width)
        y = np.linspace(0, height-1, height)
        xv, yv = np.meshgrid(x, y)
        
        # Convert to normalized coordinates (-1 to 1)
        x_norm = (xv - width/2) / (width/2)
        y_norm = (yv - height/2) / (height/2)
        
        # Calculate radial distance from center
        r = np.sqrt(x_norm**2 + y_norm**2)
        
        # Create center weight mask
        center_mask = np.exp(-r**2 / (2 * self.center_weight**2))
        
        # Modify vertical mapping
        if self.preserve_top:
            # Create smooth transition for top preservation
            top_region = (y_norm < 0)
            y_norm[top_region] = y_norm[top_region] * (1 + 
                (1 - np.abs(y_norm[top_region])) * (self.vertical_stretch - 1))
        
        # Apply transverse Mercator-like projection
        x_projected = x_norm * (1 + (1 - center_mask) * 0.2)
        y_projected = y_norm * self.vertical_stretch
        
        # Convert back to pixel coordinates
        map_x = (x_projected + 1) * width/2
        map_y = (y_projected + 1) * height/2
        
        # Ensure coordinates are within bounds
        map_x = np.clip(map_x, 0, width-1).astype(np.float32)
        map_y = np.clip(map_y, 0, height-1).astype(np.float32)
        
        return map_x, map_y

    def warp(self, img):
        """
        Apply the custom projection to an image
        """
        map_x, map_y = self.create_maps(img.shape)
        return cv2.remap(img, map_x, map_y, cv2.INTER_LINEAR)

In [ ]:
def test_projections(image_path):
    """
    Test different projection parameters
    """
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Could not read image: {image_path}")

    # Test different parameters
    configs = [
        {'vertical_stretch': 1.2, 'center_weight': 0.7, 'preserve_top': True},
        {'vertical_stretch': 1.3, 'center_weight': 0.8, 'preserve_top': True},
        {'vertical_stretch': 1.4, 'center_weight': 0.6, 'preserve_top': True},
        {'vertical_stretch': 1.5, 'center_weight': 0.5, 'preserve_top': True},
        {'vertical_stretch': 1.6, 'center_weight': 0.4, 'preserve_top': True},
        {'vertical_stretch': 1.7, 'center_weight': 0.3, 'preserve_top': True},
        {'vertical_stretch': 1.8, 'center_weight': 0.2, 'preserve_top': True},
        {'vertical_stretch': 1.5, 'center_weight': 0.8, 'preserve_top': True}

    ]

    results = []
    for i, config in enumerate(configs):
        projector = CustomProjection(**config)
        warped = projector.warp(img)
        results.append(warped)

    # Create comparison grid
    return results

In [ ]:
results = test_projections(image_path)

for i in results:
    plt.figure(figsize=(30, 30))
    plt.imshow(i)
    plt.show()

In [ ]:
vertical_stretch = 1.5
center_weight = 1
preserve_top = False
img = cv2.imread(image_path)

projector = CustomProjection(
    vertical_stretch=vertical_stretch,
    center_weight=center_weight,
    preserve_top=preserve_top
)
warped = projector.warp(img)
plt.figure(figsize=(30, 30))
plt.imshow(warped)
plt.title(f"{vertical_stretch} - {center_weight} - {preserve_top}")
plt.show()